[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds-reboot/python-first-part/blob/main/notebooks/hometasks/Task3_dmrf_OOP.ipynb)

# Задание
Реализовать с помощью объектно-ориентированного подхода предыдущие 2 задания. Создайте для каждой из задач отдельный класс, который позволяет ее решить.

* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe, а также в excel, pickle, БД

* Проверить состояние датафрейма и привести его в формат, позволяющий дальнейшее исследование данных

*  Сделать визуализацию для мини-исследования рынка строящейся недвижимости в одном или нескольких регионах с помощью pandas, matplotlib, seaborn, plotly и других инструментов.



In [12]:
import requests
import pandas as pd
from sqlalchemy import create_engine
import plotly.express as px


In [11]:
# !pip install plotly

  Using cached plotly-5.10.0-py2.py3-none-any.whl (15.2 MB)


## Список объектов

In [13]:
offset_ = 1
limit_ = 1000
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()

# Данные по определенному объекту

In [14]:
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/36774'

In [15]:
class DomIdLoader:
    def __init__(self, offset = 1, limit = 1000):
        self.offset_ = offset
        self.limit_ = limit
        self.url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object'
        self.objects_data = None
        self.objids = []

    def get_ids(self):
        #for i in range(10)
        paramz = {
            'offset': self.offset_,
            'limit': self.limit_,
            'sortField':'devId.devShortCleanNm',
            'sortType':'asc',
            'objStatus':'0',
        }
        res = requests.get(self.url, params=paramz)
        self.objects_data = res.json()
        objects_list = self.objects_data.get('data').get('list')
        self.objids += [x.get('objId') for x in objects_list]
        return self.objids
    def show_ids(self):
        print(self.objids)


In [16]:
class ObjectInfoExtractor:
    def __init__(self, ids):
        self.object_data = None
        self.ids = ids
        self.df = None

    def load_data(self):
        self.object_data = [requests.get(f'https://наш.дом.рф/сервисы/api/object/{x}').json().get('data') for x in self.ids]

    def df_converter(self):
        self.df = pd.json_normalize(self.object_data)
    def show_df(self):
        print(self.df)
    def df(self):
        return self.df

In [17]:
class Saver:
    def __init__(self, data, name = "Task3"):
        self.data = data
        self.name = name

    def save_csv(self):
        self.data.to_csv(self.name + ".csv")

    def save_xl(self):
        self.data.to_excel(self.name + ".xlsx")

    def save_sql(self, connection, table_name = 'objects'):
        self.data.to_sql(table_name, con=connection)

class Visualizer:
    def __init__(self, data):
        self.data = data

    def make_boxplot(self):
        fig = px.box(self.data)
        fig.show()

    def make_heatmap(self, name_x, name_y):
        fig = px.density_heatmap(self.data, x=name_x, y=name_y, marginal_x="histogram", marginal_y="histogram")
        fig.show()

In [18]:
OurHouse = DomIdLoader(limit=100)
OurHouse.get_ids()
OurHouse.show_ids()

[13383, 13397, 13399, 13398, 13391, 36912, 44076, 27476, 10891, 39176, 39174, 39175, 27049, 41421, 41420, 30292, 19457, 18708, 18709, 21534, 21536, 21535, 21537, 42192, 27542, 20524, 24093, 23862, 27806, 27808, 27801, 23864, 27802, 23863, 27803, 27798, 21720, 45249, 33899, 17712, 17707, 33745, 44621, 47610, 46571, 44489, 15885, 15886, 15888, 15889, 15891, 15890, 35324, 41620, 41615, 41613, 41617, 41614, 45256, 41618, 41616, 41612, 24819, 24818, 41621, 41619, 45772, 18136, 44752, 37465, 33525, 3148, 33217, 33224, 33161, 33139, 33186, 33206, 33175, 33230, 33183, 33167, 33152, 33172, 33188, 33165, 33227, 33177, 33101, 33174, 33210, 33173, 33197, 33201, 33200, 33098, 33187, 33237, 33166, 33184]


In [19]:
OurHouseDetail = ObjectInfoExtractor(OurHouse.get_ids())

In [20]:
OurHouseDetail.load_data()

In [21]:
OurHouseDetail.df_converter()

In [22]:
OurHouseDetail.show_df()

        id   pdId  region                          address  floorMin  \
0    13383  28105      72                         г Тюмень         4   
1    13397  28141      72  г Тюмень, ул Профсоюзная, д. 56         6   
2    13399  28141      72  г Тюмень, ул Профсоюзная, д. 56         1   
3    13398  28141      72  г Тюмень, ул Профсоюзная, д. 56         7   
4    13391  28101      72         г Тюмень, ул Профсоюзная         3   
..     ...    ...     ...                              ...       ...   
195  33177  77853      77                 д Зверево, д. 83         3   
196  33096  77853      77                  д Зверево, д. 2         3   
197  33187  77853      77                 д Зверево, д. 93         3   
198  33098  77853      77                  д Зверево, д. 4         3   
199  33153  77853      77                 д Зверево, д. 59         3   

     floorMax  objElemLivingCnt objReady100PercDt wallMaterialShortDesc  \
0          20                68        2022-12-31           

In [23]:
S = Saver(OurHouseDetail.df)

In [24]:
S.save_csv()

In [25]:
S.save_xl()

In [ ]:
S.data.photoRenderDTO = S.data.photoRenderDTO.astype('string')
S.data.photoRenderDTO

In [ ]:
S.data.objectTransportInfo = S.data.objectTransportInfo.astype('string')
S.data.objectTransportInfo

In [ ]:
S.data[['metro.colors']] = S.data[['metro.colors']].astype('string')
S.data[['metro.colors']]

In [29]:
engine = create_engine('sqlite://', echo=False)

S.save_sql(engine)

engine.execute("SELECT * FROM objects").fetchall()

[(0, 13383, 28105, 72, 'г Тюмень', 4, 20, 68, '2022-12-31', 'Другое', 'По желанию заказчика', 'Есть', 31, 6293.78, '72-000095', 'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/eac27d84-4621-443d-9608-623376b6bafe', '07-09-2022 15:16', '57.1519', '65.5536', 1, 0, 'Элитный', '04-10-2022 01:30', "[{'objId': 13383, 'pdId': 28105, 'rpdId': 626936, 'devId': 306, 'ordNum': 1, 'objRenderPhotoSize': 1433738, 'objRenderPhotoUrl': 'https://xn--80az8a. ... (2823 characters truncated) ... enderPhotoNm': 'IMG_20210512_0951216.jpg', 'objReadyDesc': 'Строится', 'objBuildTypeShortDesc': 'Жилое', 'knFlg': 1, 'loadDttm': '04-10-2022 01:14'}]", 0, 'normal', 'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/0dca6482-02ba-4d56-a861-7a17db04c957', 1, 39024, 20, 68, '6293.78', 10, 0, 1, '28-02-2023 12:00', '2,8', 0, 1.0, 1.0, 1.0, 0.0, 11.0, 1, 1, 0.0, 5.0, 1.0, 7, 0.407163898325013, 53109.0, 'ООО Транспромжилстрой-2005 (ИНН: 7204097730)', 1, '[]', 0, 'info@2mengroup.ru', 0, 'B', 10, 306, '2МЕН ГРУПП', 

In [48]:
V = Visualizer(OurHouseDetail.df[['floorMax']])

In [49]:
V.make_boxplot()

In [42]:
V1 = Visualizer(OurHouseDetail.df[['floorMax','objPriceAvg']])
V1.make_heatmap(name_x="floorMax", name_y="objPriceAvg")